In [1]:
import os

In [ ]:
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationandClusteringConfig:
    root_dir: Path
    data_path: Path
    metrics_file_name: Path


In [3]:
from src.CardVista.constants import *
from src.CardVista.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_and_clustering_config(self)->DataTransformationandClusteringConfig:
        config = self.config.data_transformation_and_clustering
        create_directories([config.root_dir])

        data_transformation_and__clustering_config=DataTransformationandClusteringConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            metrics_file_name= config.metrics_file_name

        )

        return data_transformation_and__clustering_config

ModuleNotFoundError: No module named 'src'

In [ ]:
from src.CardVista.logging import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score
from src.CardVista.utils.common import read_yaml, create_directories, save_json

class ReplaceOutliers(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self 
    
    

    def transform(self, X):
        schema_filepath = SCHEMA_FILE_PATH
        schema = read_yaml(schema_filepath)
        df = pd.DataFrame(X, columns=schema.COLUMNS)
        
        for col in list(df.columns):
            q1 = df[col].quantile(0.15)
            q3 = df[col].quantile(0.85)
            iqr = q3 - q1
            
            upper_limit = q3 + 1.5 * iqr
            lower_limit = q1 - 1.5 * iqr
            
            df.loc[df[col] > upper_limit, col] = upper_limit
            df.loc[df[col] < lower_limit, col] = lower_limit
            
        return df.to_numpy()

class DataTransformationandClustering:
    def __init__(self, config: DataTransformationandClusteringConfig):
        self.config = config
        
    def eval_metrics(self,data,labels):
        silhouette = silhouette_score(data, labels)
        calinski_harabasz = calinski_harabasz_score(data, labels)
        davies_bouldin = davies_bouldin_score(data, labels)

        return silhouette,calinski_harabasz ,davies_bouldin



    def data_transformation_and_clustering(self):
        logger.info("data transformation started")
        data = pd.read_csv(self.config.data_path)
        data.drop('CUST_ID',axis=1,inplace=True)
        temp = data.copy()
        replace_outliers = ReplaceOutliers()

        trf1 = SimpleImputer(strategy='median')
        trf2 = replace_outliers
        trf3 = StandardScaler()
        trf4 = KMeans(n_clusters=3,random_state = 42,n_init = "auto")

        pipe = Pipeline([('trf1',trf1),
                 ('trf2',trf2),
                 ('trf3',trf3),
                 ('trf4',trf4)
                ])
        logger.info("data fitted in pipeline")
        pipe.fit(data)


        partial_pipe = Pipeline([('trf1', trf1), 
                         ('trf2',trf2),
                         ('trf3', trf3)])
        
        logger.info("data transformed in partial pipeline")
        scaled_data = partial_pipe.transform(temp)


        labels = pipe.fit_predict(data)


        creditcard_df_scaled=scaled_data
        labels = labels

        (silhouette,calinski_harabasz,davies_bouldin) = self.eval_metrics(scaled_data,labels)
        scores = {"silhouette_score" : silhouette,"calinski_harabasz_score" : calinski_harabasz,"davies_bouldin_score" : davies_bouldin}
        save_json(path=Path(self.config.metrics_file_name),data=scores)
        logger.info("KMeans evalution metrics saved")
    
        data['CLUSTER'] = labels

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

        

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_and_clustering_config = config.get_data_transformation_and_clustering_config()
    data_transformation_and_clustering_config = DataTransformationandClusteringConfig(config=data_transformation_and_clustering_config)
    data_transformation_and_clustering_config.data_transformation_and_clustering()
except Exception as e:
    raise e

[2024-02-16 19:34:40,953 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-02-16 19:34:40,953 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-02-16 19:34:40,958 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-16 19:34:40,961 : INFO : common : created directory at: artifacts]
[2024-02-16 19:34:40,961 : INFO : common : created directory at: artifacts/data_transformation]
[2024-02-16 19:34:40,962 : INFO : 26221436 : data transformation started]
[2024-02-16 19:34:40,994 : INFO : 26221436 : data fitted in pipeline]
[2024-02-16 19:34:41,018 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-16 19:34:41,092 : INFO : 26221436 : data transformed in partial pipeline]
[2024-02-16 19:34:41,094 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-16 19:34:41,158 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-16 19:34:44,006 : INFO : common : json file saved at: artifacts